# 사전 사용 패키지 다운로드

In [1]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

# 대회 데이터 다운로드 및 사전 확인

In [2]:
train = pd.read_csv("train.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기

train.head(2)

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0


In [3]:
train.shape

(295945, 4)

In [4]:
train['smishing'].value_counts()

0    277242
1     18703
Name: smishing, dtype: int64

In [5]:
train[train['smishing'] == 1]

,id,year_month,text,smishing
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준...,1
23,26,2017-01,(광고)XXX추가 XXX품 특판 안내문XXX 지점에서 취급하고 있는 여신 XXX...,1
31,36,2017-01,(광고)XXX신용관리 XXX 알고 싶다나의 신용과 재무상태는 직접 관리해야지 누군가...,1
40,45,2017-01,(광고)안녕하세요수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA ...,1
50,58,2017-01,(광고)한국citi bank 나의 대출한도와 금리는? 대출때문에 고민하고 있다거나 ...,1
...,...,...,...,...
295927,336357,2018-12,(광고)XXX금전을 요구하는건 사기입니다. 유의하세요!1.부득이하게 높은 이자를 내...,1
295932,336362,2018-12,(광고)XXX금전을 요구하는건 사기입니다. 유의하세요!1.부득이하게 높은 이자를 내...,1
295934,336366,2018-12,(광고)XXX나의 한도금리를 알아보자금전을 요구하는건 사기입니다. 유의하세요!1.부...,1
295936,336369,2018-12,(광고) XXX항상 (XXX국민)은행을 이용해주셔서 감사합니다. 2018년 12월 ...,1


In [6]:
test = pd.read_csv("public_test.csv")

test.head(2)

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...


In [7]:
submission=pd.read_csv("submission_제출양식.csv")

submission.head(2)

,id,smishing
0,340000,1.000000e-09
1,340001,1.000000e-09


In [8]:
train.shape, test.shape, submission.shape

((295945, 4), (1626, 3), (1626, 2))

총 train data의 행은 약 29만개, test data의 행은 약 1600개로 되어있다.

## train data under-sampling method

In [9]:
Counter(train['smishing'])

Counter({0: 277242, 1: 18703})

In [10]:
19646/train.shape[0]

0.0663839564783997

NLP 특성상 해당 train data를 모두 tokenizing 한 뒤, CountVectorizer를 하게 되면,

 colab 상에서는 과하게 ram을 잡아먹어 다운이 되는 경우가 생기기 때문에, 스미싱 비율에 맞게 under sampling을 수행하였다.

In [11]:
850/12600

#  smishing:850개  non-smishing :12600개       , 비율 0.06:0.94

0.06746031746031746

In [12]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(train[train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 11750 )

random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [13]:
# random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
# train_nsm_list=list(train[train['smishing']!=1].index)
# train_nsmishing=random.sample(train_nsm_list, 11750*7 )

# random.seed(2019)
# train_sm_list=list(train[train['smishing']==1].index)
# train_smishing=random.sample(train_sm_list, 850*7 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

# 성능 저하됨

In [14]:
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

train_yy=DataFrame(train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

In [15]:
test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])

In [16]:
train_xx.shape,train_yyy.shape,test_xx.shape,test_yyy.shape

((12600, 4), (12600, 1), (1626, 1), (1626, 1))

In [17]:
# konlpy 설치
# pip install konlpy

# Mecab 설치
# http://konlpy.org/en/latest/install/
# https://cleancode-ws.tistory.com/97

import konlpy
from konlpy.tag import Mecab

tokenizer = Mecab('C:\mecab\mecab-ko-dic') # setting tokenizer using Mecab()

In [18]:
train_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( train_xx['text'], train_yyy['smishing'] ) )  ] # Mecab를 활용하여 text를 토큰화 시킴
test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) )  ]

12600it [00:02, 4826.96it/s]
1626it [00:00, 3216.35it/s]


In [19]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

필요없는 단어들을 제거하면서 동시에 모형에 사용하기 위한, X_train,test, Y_train,test를 생성한다

In [20]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_train.append(" ".join(temp))

In [21]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

CountVecotrizer를 사용하여서 학습 데이터를 바탕으로 벡터화를 시키고 이를 test data에 적용한다

In [22]:
v=CountVectorizer()

v.fit(X_train)

vec_x_train= v.transform(X_train).toarray()
vec_x_test= v.transform(X_test).toarray()

naive bayes중 MultinomialNB를 사용하여서 학습 모형을 만들고 이를 test data에 적용한다

In [23]:
m1= MultinomialNB()
m1.fit(vec_x_train,Y_train)

y_train_pred1=m1.predict_proba(vec_x_train)
y_train_pred1_one= [ i[1]  for i in y_train_pred1]

y_test_pred1=m1.predict_proba(vec_x_test)
y_test_pred1_one= [ i[1]  for i in y_test_pred1]

test data에서 각 text당 스미싱 확률을 구할 수 있으니 이를 제출양식의 smishing 변수에 넣고 csv 파일로 내보낸다

In [22]:
submission['smishing'] = y_test_pred1_one

submission.to_csv("2019-12-06-Smishing_12_12.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

이제 csv 파일로 나온 답지 파일을 14회 대회 제출 게시판을 통해 제출하시면 됩니다